<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900">**4:** 用于 Seq2Seq 的编码器-解码器结构</font>

在之前的 notebook 中，我们研究了仅使用编码器的“BERT”类模型适合做的一些任务。亲眼目睹了语言模型的强大能力，但也注意到有一些关键的限制，即单 token 或单段落的推理限制了模型在某些任务中的发挥。零样本分类 pipeline 通过多次查询编码器为每个类生成预测来解决这一问题。在本 notebook 中，我们将把这个模式扩展为一个可以用类似方式生成有序序列的架构组件，并用它生成没有限制的回复。

#### **学习目标：**

* 了解编码器-解码器模型，这些模型用编码器对静态的上下文（即问题、指令等）进行编码，用解码器预测单词。
* 了解用于创建能完成各项任务的大型通用模型的策略。

---

## 4.1. 机器翻译任务

[**机器翻译**](https://huggingface.co/tasks/translation)是一个常用术语，用于描述使用软件自动将一种语言翻译成另一种语言的任务。是的，它的定义可能没有那么严格，但在当今是一项极为重要的任务！

> <div><img src="imgs/task-translation.png" width="800"/></div>
>
> **来源: [Translation Task | HuggingFace](https://huggingface.co/tasks/translation)**

在我们可以用大语言模型解决的一系列问题中，它就是一个**序列到序列（seq2seq）**的实例。

之前我们了解到，在满足以下条件之一时，类 BERT 的编码器架构可以用来解决简单的问题：

* 输入和输出序列的条目数相同时，可通过对每个输入 token 进行预测来求解。
* 输出序列是输入序列的子集时，可通过范围预测求解。

您也可以创造性的使用这一范式，去掉这个架构的一些硬性限制，但仍会被认为是编码器的衍生。

我们经常使用**编码器**这个术语，但您可能还没意识到它为什么这么叫。考虑到自动编码器的逻辑，您可能凭直觉*（合理但并不完全正确的）*认为编码器是根据任务订制的一系列 transformer 块，其输出是一种人类不可读的隐编码。以这种范式理解的话，分类头就是用来生成人类可读输出的“解码器”。

这确实很合理，但并不是语言建模社区成员们通常谈论它的方式。

* **自动编码器逻辑：**输入样本 $\to$ 隐式序列 $\to$ 样本洞察/重建
* **语言模型逻辑：**输入段落 $\to$ 隐式序列 $\to$ 响应序列

根据该逻辑，我们或多或少可以看到，在类似 BERT 的架构中没有自然的**响应**，只有对数据的洞察。当然，洞察的形式或许是能够生成回复的基于输入的概率分布，但我们**并没有在生成新的 token**。

**这个时候，就需要解码器了！**

## 4.2. 拉取 T5 模型

我们已经提到了哪些任务无法用编码器模型完成，即机器翻译这种 seq2seq 任务，所以我们可以先找一个这类模型，看看它是怎么工作的！当然，我们可以在 HuggingFace 模型库中找到这样一个模型，但这次我们试试从[机器翻译任务页面](https://huggingface.co/tasks/translation)找找看。通过阅读，您会注意到一些建议和概述，还可以看到它提供了 pipeline 支持！事实上，如果您不想的话，甚至不需要指定背后的模型：`transformers` 将为您做出选择（不过现在我们要自己来选）：

In [ ]:
from transformers import pipeline

translator = pipeline('translation_en_to_fr', model='t5-base')
translator("Hello World! How's it going?")

我只想说：“哇！它真的管用！你能相信它起作用了么？”，但实际上这已经是可以预见的了。您已经开始预期大语言模型可以在很多任务上给出很好的结果，如果您之前用过 Google 翻译，那这或许并不那么令人惊讶。但看到它仅需要这么少的参数就能完成任务还是很令人欣慰的：

```python
(
    translator.model.name_or_path,     ## 't5-base'
    translator.model.num_parameters()  ## 222,903,552
)
```

接下来，我们来详细了解一下这个 pipeline：

In [ ]:
text_en = "Hello World! How's it going?"
resp_fr = translator("Hello World! How's it going?")
text_fr = resp_fr[0]['translation_text']

tok = translator.tokenizer
tokens_ins = [tok.decode(x) for x in tok.encode(text_en)]
tokens_out = [tok.decode(x) for x in tok.encode(text_fr)]
print(f"Inputs of length {len(tokens_ins)}: {' | '.join(tokens_ins)}")
print(f"Output of length {len(tokens_out)}: {' | '.join(tokens_out)}")

您可能会注意到以下几个有趣的结果：

* 从表面上看，您可能会注意到一个新的 `</s>` token。这定义了一些关于数据长度的信息，在训练的时候是必须带上的。
* 更特别的是 **token 的数量不匹配！**输入中有 10 个 token，但输出中有 13 个，这是怎么做到的？

除了您指定的输入之外，pipeline 可能还拿到了一些其它东西。 `preprocess` 的输出直接输入到了 `model.forward`，我们来看看具体发生了什么：

In [ ]:
prep = translator.preprocess
tokens_in2 = [tok.decode(x) for x in prep(text_en)['input_ids'][0]]
print(f"Model Inputs of length {len(tokens_in2)}: {' | '.join(tokens_in2)}")
tok.decode(prep(text_en)['input_ids'][0])

好的！模型实际上输入了更多的 token，开头的部分实际上是一条指令。所以这是怎么回事？为了弄清楚，我们来深入了解一下模型架构。

## 4.3. 解释 T5 架构

让我们继续调查模型，看看能不能搞清楚发生了什么！您可以试试以下操作：

```python
translator.model           ## See that there's a lot of stuff going on here
translator.model.encoder   ## See that this looks a lot like the BERT model
translator.model.decoder   ## See that this looks roughly the same and wonder what changed
```

但这将产生大量文本，有点难以阅读。既然这样，我们试试用更精简的图表格式将其可视化。下面的内容很重要，其中箭头表示了它们实际上是如何连接的。

> **注意：**在模型架构的打印输出中，编码器和解码器的第 0 个 transformer 层与第 1 至第 11 个 transformer 是分开显示的。这是由于 T5 纳入了[相对位置编码](https://paperswithcode.com/method/relative-position-encodings)，尽管在整个架构中都存在，但只在第 0 层作为一个单独的组件出现。您可以忽略这一区别。


<div><img src="imgs/t5-architecture.png" 
     alt="Encoder-Decoder Architecture"
     width="1200"/></div>

**有两个直观的区别：**

* 解码器其实是一个针对特定任务训练过的编码器，这个任务是：**给定输入序列，生成序列中的下一个 token**。在编码器-解码器架构中，模型通常从输入一个起始 token 开始（即 `<s>`），然后一次生成一个 token，直到字符串结束 token 出现（即 `</s>`）。（*实际上*它不仅仅只是一个编码器，不过具体的细节对本课程来说并不关键）
* 编码器-解码器架构允许将一些中间值注入到解码器的注意力机制中。这就叫交叉注意力，与自注意力遵循完全相同的逻辑：**用于向模型输入上下文信息的轻量级接口**。

> **数学运算原理：**假设您有来自编码器的 querys/values $K_{1..m}$/$V_{1..m}$，和来自解码器的 key $Q_{1..n}$。

> * 如果 $K_i$ 和 $Q_i$ 有相同的嵌入维度，那么 $Q_iK_i^T$ 就是一个 $n\times m$ 维的矩阵，做 softmax 之后也是如此。换言之：
 $$\text{Attention}(K_{1..m}, Q_{1..n}) \text{ 的维度是 } n\times m$$

> * 既然 $V_{1..m}$ 与 $n\times m$ 的注意力矩阵是乘法相容的：
$$\text{Attention}(K_{1..m}，Q_{1..n}) \times V_{1..m} \text { 是 } n\times d \text{ 的，} d \text{ 代表 } V_i \text{ 的维度 }$$ 

> * 所以，我们仅通过注意力机制，就将 $m$ 个元素的序列作为了 $n$ 个元素序列的上下文！重复多次，就拥有了基于上下文的强大生成能力。

最终就得到了一个有以下两个关键功能的架构：

* 通过解码器架构自回归的一个接一个的生成 token，每个新生成的 token 都被加入用于预测后续 token 的输入中。
* 频繁的将上下文信息从编码器注入解码器，确保生成的内容与总体目标保持一致。

### 观察 Token 生成的实际过程

现在我们大致了解了模型的工作原理，接下来探索一下可以用它做什么，看看它实际上是如何工作的。之前，我们看到了经典的英语到法语翻译任务，这个任务包含一些众所周知的很好的数据集，并且长期以来一直是 NLP 社区的重头戏。正如我们在深入代码时看到的，翻译任务的 pipeline 实际上只是向编码器输入加入了一句提示词作为条件，来告诉模型该做什么，但我们可以通过调用 `text2text-generation` pipeline 获得更通用的能力。这样一来我们就能完全控制上下文，可以让模型做些其它任务试试了。

In [ ]:
from transformers import pipeline

t5_pipe = pipeline("text2text-generation", model="t5-base")
t5_pipe("translate English to German: Hello world, and welcome to my notebook!")

首先来验证一下我们之前关于解码器的描述是否正确。能不能确认这个模型的回复是由一个一个的 token 构成的？我们可以阅读源代码，也可以通过插入一个回调函数，来观察进入编码器和解码器前向过程的内容以及它们被调用的顺序。

In [ ]:
from extras_and_licenses.forward_listener import ForwardListener

t5_pipe.model.encoder.forward = ForwardListener(t5_pipe.model.encoder, name='encoder', tokenizer=t5_pipe.tokenizer)
t5_pipe.model.decoder.forward = ForwardListener(t5_pipe.model.decoder, name='decoder', tokenizer=t5_pipe.tokenizer)
t5_pipe("translate English to German: Hello world!!")

In [6]:
ForwardListener.clear_all()

**我们会观察到一些关键的信息：**

* 编码器在处理输入上被调用了一次，给出了可供解码器使用的表示。在大多数模型中，编码器状态只需要计算一次就可以提供良好的上下文语境，这个静态的上下文信息就可以用来引导解码器。
	+ 这有助于避免“目标偏移（moving target）”问题，能够提高训练期间的稳定性。
* 解码器被要求一次生成一个 token，其中第一个 token 由起始 token 生成（在本例中是 `<pad>`）。第一次迭代完全取决于编码器的输出，而后续的 token 则共同取决于输入的编码和生成的历史。正如我们之前看到的，当标志着字符串结束的 token（即 `</s>`）被计算出来时，生成过程就停止了。
	+ 您可能还会注意到，解码器一次仅接收一个词。高效的实现会把之前生成过程的计算结果存下来，因为前几次迭代的 key-value 计算可以和新 token 的计算结果一块存储。您可以在 `past_key_values` 中看到它存储着前几次迭代的注意力组件，并随着生成过程不断增大。

## 4.4. 用 T5 试验

现在我们已经了解了模型的定义方式及其作用，让我们稍微调整一下模型，看看还能做点什么！

回想一下，T5 模型经过各种任务的训练，其中一部分在下图中展示了出来：

> <div><img src="imgs/t5-pic.jpg" width="800"/></div>
>
> **来源: [Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer](https://arxiv.org/abs/1910.10683v4)**

当然，这只是我们可以参考的一部分任务，更详细的信息可以在这篇文章中找到：[Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer](https://arxiv.org/abs/1910.10683v3)。但我们可以从这开始，来试试吧！

为了避免浪费太多时间，应该提醒您 `t5-base` 可能无法达到理想的效果。而 `t5-large` 模型有更多的参数，需要更多的数据来训练，但仍然在我们的计算资源范围内，所以我们可以直接加载这个更大的模型：

In [ ]:
from transformers import pipeline

## T5-Large performs better and has reasonably-fast inference, so we can safely default to it
t5_pipe_base = t5_pipe
t5_pipe_large = pipeline("text2text-generation", model="t5-large")

print(f"""
t5-base:
 - model size: {t5_pipe_base.model.num_parameters():,} parameters
 - memory footprint: {t5_pipe_base.model.get_memory_footprint()/1e9 :.03f} GB

t5-large:
 - model size: {t5_pipe_large.model.num_parameters():,} parameters
 - memory footprint: {t5_pipe_large.model.get_memory_footprint()/1e9 :.03f} GB
""")

In [ ]:
queries = [
    "translate English to Spanish: This is good!",
    "cola sentence: The course is jumping well!",
    "stsb sentence1: The rhino grazed on the grass. sentence2: A rhino is grazing in a field.",
    ## Summarize entry pulled from https://huggingface.co/docs/transformers/tasks/summarization
    "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes.",
    ## TODO: Take a skim through the original T5 paper and pull in some other queries you think might be interesting.
    ## Maybe the SQUaD question-answeing, or whatever else catches your eye. Examples of the format are around pages 50-60.
]

t5_pipe_large(queries)

如您所见，它的效果很不错。所以如果输入适当的提示词，它也许能处理任意的任务？我们来试试吧！

In [ ]:
queries = [
    "translate English to Spanish: This is good!",
    "continue the sentence: I love walking in ",
    "continue the conversation as a helpful agent: User: Hello! Agent: ",
]
t5_pipe_large(queries)

有点意思。但还是不够强大，无法进行足够的推理来理解提示词。它或许只是记住了提示词，然后通过单词表面的相似性学到了一些关键的洞察？

也许问题的关键是模型的大小。我们直接跳过了 `t5-base` 模型，因为它在基准任务上表现并不好，或许我们也可以试试比 `t5-large` 模型更大的模型？

但遗憾的是，[`t5-xl`（至少是 `google/t5-v1_1-xl` 版本）](https://huggingface.co/google/t5-v1_1-xl)和 [`t5-xxl`](https://huggingface.co/google/flan-t5-xxl) 模型超出了我们的计算资源（我们会在之后的 notebook 里尝试克服这一点）。另外，更大的模型实际上并不能解决这个问题，仅仅是因为标准 T5 的训练任务种类仍然太少了。在这种情况下，比起更大的模型，我们需要的是经过了更通用训练过程的模型。

## 4.5. 使用 Flan-T5 进行提示工程

在课程里我们介绍了 Flan-T5 模型，尤其提到了它具有更激进的训练目标。具体来说，是让模型经过了足够多的训练强化，来学习如何将输入序列作为生动的自然语言问题来进行推理，以此获得更好的性能。

> <div><img src="imgs/t5-flan2-spec.jpg" width="1000"/></div>
>
> **来源: [Scaling Instruction-Finetuned Language Models](https://arxiv.org/abs/2210.11416v5)**

让我们拉取这个模型，看看它能做些什么！

In [ ]:
from transformers import pipeline

## T5-Large performs better and has reasonably-fast inference, so we can safely default to it
flan_t5_pipe = pipeline("text2text-generation", model="google/flan-t5-large")

In [ ]:
queries = [
    "translate English to Spanish: This is good!",
    "continue the sentence: I love walking",
    "continue the conversation as a helpful agent: User: Hello! Agent: ",
]

flan_t5_pipe(queries)

如您所见，它实际上能够遵循更多的指令并能推理出指令的实际含义，这太棒了！事实上，跟这个模型能做的事情相比，我们之前给 `T5` 模型安排的超范围的目标显得相当基础！这种涌现式的现象，即能够仅基于自然语言去推理语境来适应新任务的能力，被称为**上下文学习**，也是使[**提示工程**](https://en.wikipedia.org/wiki/Prompt_engineering)得以实现的新范式。

您可能已经听过很多关于**提示工程**的事，其核心就是弄清楚什么样的输入能让训练好的模型达到最佳的表现。实际应用中，以下的经验通常很管用：

* **格式遵循（Format Abiding）：**想想模型是怎么训练的，训练过程的输入一般都是什么格式。任务形式越接近，执行效果就越好。
* **少样本提示（Few-Shot Prompting）：**由于训练数据的关系，模型倾向于处理重复的模式，因此给一些例子可能是个好主意。这被称单样本或少样本提示，指的就是在推理时，给模型一个或几个表现良好的示例，看看模型会不会遵从这个模式。
* **迭代试错（Iterative Trial-And-Error）：**如果模型可以理解并遵循您的指令（例如经过了指令微调），您还可以通过继续更新提示来纠正模型的不理想行为。每个模型的这个过程可能很不一样，有些模型很擅长遵循指令，有些则不太擅长，有些则无法偏离训练任务类型去完成另外一项任务。
* **启动（Priming）：**除了给出“指令”，您还可以将回复内容的开头直接输给解码器，让它接着往下生成。这是一种绕过默认行为的很好的方式（结果或好或坏），不过默认的编码器-解码器架构可能并不支持这种行为。

了解了这些，看看您可以用这个模型做些什么，可以通过下面的任务再熟悉一下它的能力：

### **任务 1：按预期执行**

向网络提问，看看它能做什么。您可以用冒号来分开任务和正文，也可以跟模型直接交流。下面是一些有启发性的例子，也可以用你自己的想法来试试！

In [22]:
# flan_t5_pipe("Can you tell me about how sandwiches are made?")
# flan_t5_pipe("Is this a true sentence: You can make fire by rubbing two sticks together very quickly")
# flan_t5_pipe("How do you say 'when in Rome...' in french?")
# flan_t5_pipe("Identify the noun with negative sentiment: I love pizza with pineapple, but adding pickles to it is just too much")
# flan_t5_pipe("Translate english to pig latin: Hello world and all who live in it!") ## will fail

现在，还用这几个指令，不过直接询问模型。看会不会影响它完成任务，或许它能适应的比较好。当它没法完成任务时，看看你能不能把它修好。

**提示：**
* 如果需要的话，您可以考虑给模型正确执行的示例。这就是少样本提示，您在告诉它应该如何表现。
* 对于 Pig Latin 任务，除非您真的很走运，否则是不可能实现的。这个任务需要以不自然的语义形式进行字母级的推理，因此您的所有提示工程尝试可能都会失败。

In [ ]:
## Example of few-shot prompting. Will still fail though, due to the above reason
'''
flan_t5_pipe("""
Translate english to pig latin.
English: Look in the bag!
Pig Latin: Ooklay in the agbay!
English: Hello world and all who live in it!
Pig Latin: """)
'''

# flan_t5_pipe("Answer the question like a dictionary: Can you tell me about how sandwiches are made?")
# flan_t5_pipe("Is the sentence 'You can make fire by rubbing two sticks together very quickly' true? Please explain")

注意，对于 Pig Latin 任务，您可能无法让它成功。这可能与分词策略有关：虽然它理应能表示 Pig Latin，但在未经特殊微调的情况下，这个任务还是有点超范围了。

### **任务 2：引发幻觉问题**

询问模型不知道的问题时，您有没有注意到模型产生了幻觉（Hallucination）？您认为应该怎么解决这些问题？

In [31]:
# flan_t5_pipe("What is my name?")
# flan_t5_pipe("Who are you?")
# flan_t5_pipe("How do you know my name?")
# flan_t5_pipe("Where do you work?")
# flan_t5_pipe("Answer the question honestly: Who are you?")

### **任务 3：开始聊天**

让模型与顾客聊天，看看它都能做些什么。挑战一下，看看您能不能把模型弄坏。或者尝试在模型中输入更多内容，看它能输出些什么！

In [ ]:
## Challenge: Try to break the model by introducing an inconsistency.
## - Possible option: Make the role "Human" instead of "Customer"
## - Possible option: See if you can get the model to behave awkwardly by only changing the user input

''' # Example
flan_t5_pipe("""
You are a friendly chat agent who is helping a customer.
You are supposed to be nice and helpful, and tried to answer in detail.
If you do not know something, say "I don't know". Do not lie!
Customer: Hello! How's it going? Who are you?
Agent: """)
'''

## Example of breaking the model
flan_t5_pipe("""
You are a friendly chat agent who is helping a customer.
You are supposed to be nice and helpful, and tried to answer in detail.
If you do not know something, say "I don't know". Do not lie!
Human: Hello! How's it going? Who are you?
Agent: """)

## 4.6. **总结：**技术回顾

现在，我们已经看到了语言模型是如何通过将语言编码作为语境来生成全新文本的。这开辟了许多新的可能性，也留下了许多尚未解决的问题，但至少我们现在已经处于前沿了，能用有限的算力完成一些非常强大的任务！下一节，我们将看到编码器-解码器真正发挥作用的地方：**多模态生成**。再之后，我们将讨论对解码器来说更适合的任务：**文本生成**。

In [ ]:
## Please Run When You're Done!
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)